<img style="float: left; margin: 30px 15px 15px 15px;" src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTFzQj91sOlkeDFkg5HDbjtR4QJYmLXkfMNig&usqp=CAU" width="400" height="500" />
    
    
## <font color='Black'> Deep Learning.

###Proyecto 1 - Autoencoders.

**Nombre:** Sergio Daniel Dueñas Godinez.

**Expediente** : 739300.
    
**Profesor:** Iván Reyes Amezcua.
    
**Link Github**: https://github.com/SergioDuenass/Camus-GPT

In [1]:
# Librerias a utilizar
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.losses import binary_crossentropy, mean_squared_error
from tensorflow.keras.layers import Input, Dense, Flatten, Reshape, Conv1D, Conv1DTranspose, Lambda

import numpy as np
import spacy
import re


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Tokenización, embeddings y limpieza de corpus

In [3]:
def clean_corpus_from_file(file_path):
    cleaned_corpus = []

    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            # Leer el contenido del archivo
            corpus = file.readlines()

            for text in corpus:
                # Convertir a minúsculas
                text = text.lower()

                # Eliminar caracteres no alfabéticos y números
                text = re.sub(r'[^a-z\s]', '', text)

                # Eliminar espacios en blanco adicionales
                text = ' '.join(text.split())

                cleaned_corpus.append(text)

    except Exception as e:
        print(f"Error al leer el archivo: {e}")

    return cleaned_corpus



def clean_corpus(corpus):
    cleaned_corpus = []

    for text in corpus:
        # Convertir a minúsculas
        text = text.lower()

        # Eliminar caracteres no alfabéticos y números
        text = re.sub(r'[^a-z\s]', '', text)

        # Eliminar espacios en blanco adicionales
        text = ' '.join(text.split())

        cleaned_corpus.append(text)

    return cleaned_corpus

In [4]:
# Utilizo spacy para la tokenización
nlp = spacy.load("en_core_web_sm")

def tokenize_corpus(corpus):
    tokenized_corpus = []

    for text in corpus:
        # Tokenizar el texto usando spaCy
        tokens = [token.text for token in nlp(text)]
        tokenized_corpus.append(tokens)

    return tokenized_corpus

In [5]:
corpus_file_path = "/content/drive/MyDrive/ProyectoCamus/data/corpus.txt"
cleaned_corpus = clean_corpus_from_file(corpus_file_path)

In [6]:
tokenized_corpus = tokenize_corpus(cleaned_corpus)

In [7]:
# Construir un vocabulario
vocabulario = {token: idx for idx, token in enumerate(set(token for sublist in tokenized_corpus for token in sublist))}
vocab_size = len(vocabulario)

# Convertir tokens a índices
corpus_indices = [[vocabulario[token] for token in secuencia] for secuencia in tokenized_corpus]

# Padding de las secuencias
corpus_padded = pad_sequences(corpus_indices, padding='post', truncating='post')

# Crear el modelo de embedding
embedding_dim = 300  # ajusta según tus necesidades
embedding_model = Embedding(input_dim=vocab_size, output_dim=embedding_dim)

# Obtener vectores de embedding
embedded_sequence = embedding_model(corpus_padded)

# Ver la salida
print("Corpus Padded Shape:", corpus_padded.shape)
print("Embedded Sequence Shape:", embedded_sequence.shape)

Corpus Padded Shape: (8289, 106)
Embedded Sequence Shape: (8289, 106, 300)


# VAE

In [8]:
embedded_sequence.shape

TensorShape([8289, 106, 300])

In [9]:
input_shape = embedded_sequence.shape[1:]
batch_size = 128
latent_dim = 64
epochs = 10

In [10]:
def sampling(args):
  z_mean, z_log_var = args

  dim = K.int_shape(z_mean)[1]

  # TODO: check dimensions
  epsilon = K.random_normal(shape = (K.shape(z_mean)[0], dim))

  return z_mean + K.exp(0.5 * z_log_var) * epsilon

In [11]:
# Ajustar las dimensiones de las secuencias de entrada
target_length_input = 108
corpus_padded_input = pad_sequences(corpus_indices, maxlen=target_length_input, padding='post', truncating='post')

# Convertir a embeddings
embedded_sequence_input = embedding_model(corpus_padded_input)

# Ajustar las dimensiones de las secuencias de salida
target_length_output = 108
corpus_padded_output = pad_sequences(corpus_indices, maxlen=target_length_output, padding='post', truncating='post')


# Convertir a embeddings
embedded_sequence_output = embedding_model(corpus_padded_output)



In [12]:
# Ajustar el modelo Encoder
input_shape_encoder = (target_length_output, embedding_dim)  # Ajusta según tus necesidades
latent_dim = 32  # Ajusta según tus necesidades

inputs_encoder = Input(shape=input_shape_encoder, name="encoder_input")
x_encoder = Conv1D(32, 3, activation="relu", strides=2, padding="same")(inputs_encoder)
x_encoder = Conv1D(64, 3, activation="relu", strides=2, padding="same")(x_encoder)

shape_before_flat_encoder = K.int_shape(x_encoder)
x_encoder = Flatten()(x_encoder)
x_encoder = Dense(256, activation="relu")(x_encoder)

z_mean_encoder = Dense(latent_dim, name='z_mean')(x_encoder)
z_log_var_encoder = Dense(latent_dim, name='z_log_var')(x_encoder)

encoder = Model(inputs_encoder, [z_mean_encoder, z_log_var_encoder], name='encoder')

# Ajustar el modelo Decoder
latent_inputs_decoder = Input(shape=(latent_dim,), name='z_sampling')
x_decoder = Dense(np.prod(shape_before_flat_encoder[1:]), activation="relu")(latent_inputs_decoder)
x_decoder = Reshape(shape_before_flat_encoder[1:])(x_decoder)
x_decoder = Conv1DTranspose(64, 3, activation="relu", strides=2, padding="same")(x_decoder)
x_decoder = Conv1DTranspose(32, 3, activation="relu", strides=2, padding="same")(x_decoder)
outputs_decoder = Conv1DTranspose(embedding_dim, 3, activation="sigmoid", padding="same")(x_decoder)

decoder = Model(latent_inputs_decoder, outputs_decoder, name='decoder')

In [13]:
# VAE
outputs_vae = decoder(encoder(inputs_encoder)[0])
vae = Model(inputs_encoder, outputs_vae, name='vae')

# Compilar el modelo con la función de pérdida personalizada
vae.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredError())

# Resumen del modelo
vae.summary()


Model: "vae"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_input (InputLayer)  [(None, 108, 300)]        0         
                                                                 
 encoder (Functional)        [(None, 32),              494112    
                              (None, 32)]                        
                                                                 
 decoder (Functional)        (None, 108, 300)          104652    
                                                                 
Total params: 598764 (2.28 MB)
Trainable params: 598764 (2.28 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
# Entrenar el modelo
vae.fit(embedded_sequence_input, embedded_sequence_output, epochs=epochs, batch_size=batch_size)

Epoch 1/10


65/65 [==============================] - 11s 43ms/step - loss: 0.0836
Epoch 2/10
65/65 [==============================] - 2s 28ms/step - loss: 8.9004e-04
Epoch 3/10
65/65 [==============================] - 2s 29ms/step - loss: 8.9004e-04
Epoch 4/10
65/65 [==============================] - 1s 22ms/step - loss: 8.9004e-04
Epoch 5/10
65/65 [==============================] - 1s 22ms/step - loss: 8.9004e-04
Epoch 6/10
65/65 [==============================] - 2s 23ms/step - loss: 8.9004e-04
Epoch 7/10
65/65 [==============================] - 1s 23ms/step - loss: 8.9004e-04
Epoch 8/10
65/65 [==============================] - 1s 23ms/step - loss: 8.9004e-04
Epoch 9/10
65/65 [==============================] - 1s 23ms/step - loss: 8.9004e-04
Epoch 10/10
65/65 [==============================] - 2s 25ms/step - loss: 8.9004e-04


In [15]:
# Obtener z_mean, z_log_var para un lote de imágenes de entrada
z_mean_batch, z_log_var_batch = encoder.predict(embedded_sequence_input)

# Utilizar la función de muestreo para generar muestras de la distribución latente
latent_samples = sampling([z_mean_batch, z_log_var_batch])

# Decodificar las muestras generadas para obtener imágenes "inventadas"
decoded_images = decoder.predict(latent_samples)


260/260 [==============================] - 1s 2ms/step


#### Cálamos el modelito

In [27]:
# Crear el vocabulario inverso
inv_vocabulario = {idx: palabra for palabra, idx in vocabulario.items()}

In [29]:
# Convertir los índices de embedding a palabras usando el vocabulario inverso
palabras_recuperadas = []

for idx_mas_cercano in decoded_images[0]:
    # Verificar si el índice está en el rango del vocabulario
    if np.all(idx_mas_cercano < len(inv_vocabulario)):
        # Si idx_mas_cercano es un array, obtener las palabras correspondientes a los índices
        if isinstance(idx_mas_cercano, np.ndarray):
            palabras_recuperadas.extend([inv_vocabulario[idx] for idx in idx_mas_cercano])
        else:
            # Si es un escalar, obtener la palabra correspondiente al índice
            palabra_recuperada = inv_vocabulario[idx_mas_cercano]
            palabras_recuperadas.append(palabra_recuperada)
    else:
        print("Índice más cercano fuera de rango del vocabulario")

# Unir las palabras para obtener la oración original
oracion_original = ' '.join(palabras_recuperadas)

# Imprimir la oración original
print("Oración Original:")
print(oracion_original)


Oración Original:
spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spell spel

Malísimo pero lo intentó.

Tengo la teoría que el error que me da, es que estoy tratando los outputs del modelo como si fueran de los embeddings que usé pero quizá los vectores que me da no son compatibles, quizá tendría que hacer uno personalizado o no sé.